<img align="left" src="NSIDC_logo_2018_web.jpg" width=120>


# Getting the most out of NSIDC DAAC data: Discovering, Accessing, and Harmonizing Arctic Remote Sensing Data

Welcome to the NSIDC DAAC's hands-on tutorial at the 2019 AGU Fall Meeting! 

#### The learning objectives of this tutorial include:

* Become familiar with NSIDC resources, including user support documents, data access options, and data tools and services.
* Learn about the coverage, resolution, and structure of sea ice data from the new NASA ICESat-2 mission.
* Learn how to access and subset data programmatically using the NSIDC DAAC's API service.
* Interact with ICESat-2 and MODIS data using basic Python science libraries to visualize, filter, and plot concurrent data.

The tutorial is divided into three main Jupyter Notebooks: __Introduction__, __Customize and Access Data__, and __Visualize and Analyze Data__. Here are the steps you will learn in this Introduction notebook:
  
1. Learn the basics of Jupyter Lab and Jupyter Notebook
2. Explore NSIDC data availability and resources
3. Get introduced to the Arctic sea ice case study
4. Visualize time and area of interest using [OpenAltimetry](http://openaltimetry.org/) 
   

 Let's get started!
 ___

# Introduction to Project Jupyter 

Before we delve into our Arctic sea ice data exploration, let's go over the basics of how to interact with these Jupyter Notebooks and the Jupyter Lab interface, as well as some background on our JupyterHub environment.

### The "classic" [Jupyter Notebook](https://jupyter-notebook.readthedocs.io/en/stable/) 

Interactive, reproducible, exploratory, open source

  * Browser integrated computing environment
  * Popular for interactive data analysis
  * Reproducible document formats combined into single notebook (code, markdown, equations, visualizations)
  * Create and share documents that contain live code, equations, visualizations and narrative text.
  * Supports over 100 programming languages 
    
### [JupyterLab](https://jupyterlab.readthedocs.io/en/stable/)

Integrated IDE for computational workflows

  * Web-based interactive development environment for Jupyter notebooks, code, and data
  * JupyterLab goes beyond the "classic" Jupyter Notebook by providing a flexble and extensible web application with a set of reusable components
  * All the building blocks for computation put into one integrated interface 
  * Allows multiple notebooks, text editors, terminals, output areas, and custom components using tabs and collapsible sidebars
  
  
### [JupyterHub](https://jupyterhub.readthedocs.io/en/stable/)

You are here! For this AGU tutorial we will be working within the NSIDC JupyterHub, powered by Amazon Web Services: https://nsidc-cryohub.org/

No need to download or install any packages. JupyterHub proxies multiple instances of a single-user Jupyter notebook server. We have preconfigued the environments and data that you'll be working with during the tutorial.

***NSIDC CryoHub will not be accessible after the week of AGU. Please visit the [AGU-2019-NSIDC-Data-Tutorial](https://github.com/nsidc/AGU-2019-NSIDC-Data-Tutorial) Github Repository and follow the README file instructions to install and run these tutorials in a Binder instance or on your local machine.***

___

## Jupyter Notebook Basics
The body of a notebook is composed of cells. Each cell contains either markdown, code input, code output, or raw text. Cells can be included in any order and edited at-will, allowing for a large ammount of flexibility for constructing a narrative.

- **Markdown cells** - These are used to build a nicely formatted narrative around the code in the document. 

- **Code cells** - These are used to define the computational code in the document. They come in two forms: the *input cell* where the user types the code to be executed, and the *output cell* which is the representation of the executed code. 

- **Raw cells** - These are used when text needs to be included in raw form, without execution or transformation.


### Execute a cell or selected cells by pressing shift + enter

In [ ]:
print('Hello World!')

### Execute a cell and insert new cell below by pressing Alt + Enter

In [ ]:
print('Hello World!')

### Bulk comment and uncomment by selecting code and pressing Ctrl + /

In [ ]:
dog = ['Amy', 1, 2.6, True, 3000, ['frisbee', 'ball']]
for i in dog:
     print(i)

### Collapse a cell or cell output by clicking on the blue line to the left of the cell 

The cell content is replaced by three dots, indicating that the cell is collapsed.


### Execute multiple cells or run the entire notebook 

 * Select cells with **shift + Up or shift + Down** and then execute selection with **shift + enter.**
 
 * Run the whole notebook in a single step by clicking on the menu **Run -> Run All Cells.**
 
 ___

## Overview of NSIDC data and resources

[The National Snow and Ice Data Center](https://nsidc.org) provides over 1100 data sets covering the Earth's cryosphere and more, all of which are available to the public free of charge. Beyond providing these data, NSIDC creates tools for data access, supports data users, performs scientific research, and educates the public about the cryosphere. 

### Data Resources

* [NSIDC Data Search](https://nsidc.org/data/search)
    * Search all NSIDC Data
* [NSIDC Data Updates Announcements](https://nsidc.org/the-drift/data-update/) 
    * News and tips for data users
* [NASA Earthdata Search](http://search.earthdata.nasa.gov/)
    * Search and access data across the NASA Earthdata
    
We'll explore data access and support resources in more detail below with our ICESat-2 and MODIS data sets of interest.

___

 ## NASA's ICESat-2 mission

<img align="right" src="https://nsidc.org/sites/nsidc.org/files/images/atlas-beam-pattern.png" width=600>

The Ice, Cloud, and Land Elevation Satellite-2 (ICESat-2) carries the Advanced Topographic Laser Altimeter System (ATLAS), a photon-counting laser altimeter, that measures the height of Earth's changing polar regions. The ATLAS instrument releases 10,000 laser pulses per second, split into 3 weak/strong beam pairs at a wavelength of 532 nanaometers (bright green on the visible spectrum). Measurements are taken every 70 cm along the satellite's ground track with a footprint diameter of 17 m. The beam pairs are spaced 3.3 km apart, with a distance of 90 m between each weak and strong beam. ICESat-2 orbits Earth every 91 days to allow scientists to track height changes in all four seasons. In addition to land ice and sea ice, ICESat-2 also measures the height of vegetation, land surface, water, and clouds. See the [NASA ICESat-2 home page](https://icesat-2.gsfc.nasa.gov/) for more information. 
 


  ### Data references 
  
 [The NSIDC ICESat-2 home page](https://nsidc.org/data/icesat-2) provides an overview of the data products and user resources available. 
 
 * Associated access, visualization, and data customization tools and services are provided on the [ICESat-2 Tools page](https://nsidc.org/data/icesat-2/tools).
     * [ICESat-2 Hackweek tutorials](https://github.com/ICESAT-2HackWeek/ICESat2_hackweek_tutorials)
     * [icepyx](https://github.com/icesat2py/icepyx): A collection of python tools for obtaining and working with ICESat-2 data.
 
 
 
Each data set has an associated landing page with citation information, a curated user guide, support documentation, and the ability to access data by time, spatial region, and granule name:


* [ATL03](https://nsidc.org/data/atl03): Global Geolocated Photon Data
     * ATL03 provides time, latitude, longitude, and ellipsoidal height for each photon event downlinked from ATLAS. Heights are corrected for several geophysical phenomena, such as effects of the atmosphere and solid earth deformation. 
     * [ATL03 Data Dictionary](https://nsidc.org/sites/nsidc.org/files/technical-references/ATL03-data-dictionary-v001.pdf)
       * Resolution: 70 cm
       * Format: HDF5; most variables are provided as 1-D arrays
* [ATL07](https://nsidc.org/data/atl07): Level 3A Sea Ice Height
     * ATL07 provides along-track surface height and type (e.g., snow-covered ice, open water) for the ice-covered seas of the northern and southern hemispheres. Sea surface and sea ice height are estimated for segments along each of the six ground tracks. 
     * [ATL07 Data Dictionary](https://nsidc.org/sites/nsidc.org/files/technical-references/ATL07-data-dictionary-v001.pdf)
         * Resolution:  Each height value corresponds to a variable along-track length made up of 150 photons.
         * Format: HDF5; most variables are provided as 1-D arrays
* [ATL10](https://nsidc.org/data/atl10): Level 3A Sea Ice Freeboard
    * Add more info...


## MODIS Ice Surface Temperature 

Moderate-resolution Imaging Spectroradiometer (MODIS) ice surface temperature (IST) data are available in Level-2 swaths and Level-3 grids. For this tutorial we will be using the Level 2 swath data. MODIS data are available at 1 km resolution. IST retrievals occur for cloud free Arctic and Antarctic sea ice, for day and night. 

IST is the radiating temperature at the ice surface. The IST algorithm for MODIS uses a split-window technique. MODIS bands 31 and 32 (brightness temperature) provide the ability for IST detection. 

### Data references

[The NSIDC MODIS home page](https://nsidc.org/data/modis/) provides an overview of the data products and user resources available.

* Associated access, visualization, and data customization tools and services are provided on the [MODIS Tools page](https://nsidc.org/data/modis/tools.html).

Each data set has an associated landing page with citation information, a curated user guide, support documentation, and the ability to access data by time, spatial region, and granule name:

* [MOD29](https://nsidc.org/data/mod29)/[MYD29](https://nsidc.org/data/myd29): Level 2 sea ice surface temperature
  * Resolution: 1 km 
  * Format: swath, HDF-EOS 2
        

## AMSR-E/2 Unifed polar brightness temperature
The AMSR-E/AMSR2 Unified Level-3 6.25 km product includes brightness temperatures at 89.0 GHz. Data are mapped to a polar stereographic grid at a spatial resolution of 6.25 km for the Northern and Southern Hemispheres. This product uses the Japan Aerospace Exploration Agency (JAXA) AMSR2 Level-1R input brightness temperatures that are calibrated (unified) across the JAXA AMSR-E and AMSR2 Level-1R products.

### Data references

* [AMSR2/U](https://nsidc.org/data/amsre_amsr2):

     * [AU_SI6](https://nsidc.org/data/AU_SI6/): polar brightness temperatures
         * Resolution: 6.25 km 
         * Format: gridded, HDF-EOS5
         

____
## Example Application: Using ICESat-2 height and MODIS IST data to investigate the boundary between shore fast ice and drift ice in the East Siberian Sea.    

### Some context...
 * **Fast ice**: *ice that is anchored (fastened) to the shore or ocean bottom, typically over shallow ocean shelves at continental margins; fast ice is defined by the fact that it does not move with the winds or currents.*
 
     * **Characteristics**:
  
        * Attached to shoreline or pinned to ocean bottom
        * Lower surface roughness compared to drift ice
        * Graduated thickness from new ice to thicker 1st year ice 
        * Leads typically separate fast ice from drift ice     
___ 

## Let's take a first glance of our data.

We are looking for a clear day across the coast, fast ice, and on to drift ice: March 23, 2019 over the East Siberian Sea provides those ideal conditions. Below is imagery overlain with MODIS Terra IST over this region:

![MODIS](https://wvs.earthdata.nasa.gov/api/v1/snapshot?REQUEST=GetSnapshot&TIME=2019-03-23T00:00:00Z&BBOX=-1095566.2289737656,918471.1511079771,593009.7710262344,2165703.151107977&CRS=EPSG:3413&LAYERS=MODIS_Terra_CorrectedReflectance_TrueColor,Coastlines,MODIS_Terra_Ice_Surface_Temp_Day&WRAP=day,x,day&FORMAT=image/png&WIDTH=3298&HEIGHT=2436&ts=1574699150030)
NASA worldview imagery overlain with MODIS Terra IST over East Siberian Sea, March 23, 2019. Image credit: NASA Worldview

#### How about ICESat-2 coverage?

We'll use the [OpenAltimetry](https://openaltimetry.org/data/icesat2/?annoId=119&shareKey=31a6dd4555dc1b8ccbd84fce02a63d37) application to easily visualize the coverage over our area of interest, including an overlay of MODIS reflectance imagery to observe cloud cover. The link provides a previously created data annocation that automatically zooms into our study area. These annotations can be created and shared by all users of OpenAltimetry. 

***Note that we'll be predominantly investigating ATL07 (L3A Sea Ice Height) as opposed to ATL10 (L3A Sea Ice Freeboard). The ATL07 product contain heights from the ice-covered oceans of the northern and southern hemispheres when the surface is visible (relatively cloud free) and the ice concentration is > 15%. Because we're investigating thin ice, we're expecting to see lower quality and higher noise using the freeboard data, as ATL10 only provides data when the ice concentration is >50% and 25km away from the coast.***

<!-- ![Study Area](https://wvs.earthdata.nasa.gov/api/v1/snapshot?REQUEST=GetSnapshot&TIME=2019-03-23T00:00:00Z&BBOX=-1790464,1137664,-60928,2237440&CRS=EPSG:3413&LAYERS=MODIS_Terra_CorrectedReflectance_TrueColor,Coastlines,Reference_Labels&WRAP=day,x,x&FORMAT=image/png&WIDTH=1689&HEIGHT=1074&ts=1573233401157.png) -->